In [4]:
def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)


In [8]:
record = ('Dave', 'dave@example.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = record
phone_numbers

['773-555-1212', '847-555-1212']

In [10]:
'''
星号表达式也能用在列表的开始部分。比如，你有一个公司前 8 个月销售数据的序列， 但是你想看下最近一个月数据和前面 7 个月的平均值的对比。你可以这样做：


'''
*aa,bb = [1,2,3,34,4,566,67]
aa

[1, 2, 3, 34, 4, 566]

扩展的迭代解压语法是专门为解压不确定个数或任意个数元素的可迭代对象而设计的。 通常，这些可迭代对象的元素结构有确定的规则（比如第 1 个元素后面都是电话号码）， 星号表达式让开发人员可以很容易的利用这些规则来解压出元素来。 而不是通过一些比较复杂的手段去获取这些关联的元素值。

值得注意的是，星号表达式在迭代元素为可变长元组的序列时是很有用的。 比如，下面是一个带有标签的元组序列：

In [11]:
records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
]

def do_foo(x, y):
    print('foo', x, y)

def do_bar(s):
    print('bar', s)

for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


In [14]:
#星号解压语法在字符串操作的时候也会很有用，比如字符串的分割。

line = 'nobodyfsfdafad,fdafdadaf:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')
print(uname,fields,homedir,sh)




nobodyfsfdafad,fdafdadaf ['*', '-2', '-2', 'Unprivileged User'] /var/empty /usr/bin/false


有时候，你想解压一些元素后丢弃它们，你不能简单就使用 * ， 但是你可以使用一个普通的废弃名称，比如 _ 或者 ign （ignore）。

In [22]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *ign, (*ign, year) = record
year

2012

在很多函数式语言中，星号解压语法跟列表处理有许多相似之处。比如，如果你有一个列表， 你可以很容易的将它分割成前后两部分：



In [23]:
items = [1, 10, 7, 4, 5, 9]
head, *tail = items
print(head,tail)

1 [10, 7, 4, 5, 9]


如果你够聪明的话，还能用这种分割语法去巧妙的实现递归算法。比如：

In [25]:
def sum(items):
    head,*tail = items
    return head+sum(tail) if tail else head
sum(items)

36

1.4 查找最大或最小的 N 个元素

问题

怎样从一个集合中获得最大或者最小的 N 个元素列表？

解决方案

heapq 模块有两个函数：nlargest() 和 nsmallest() 可以完美解决这个问题。



In [28]:
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3, nums)) # Prints [42, 37, 23]
print(heapq.nsmallest(3, nums)) # Prints [-4, 1, 2]


[42, 37, 23]
[-4, 1, 2]


In [30]:
heapq.nlargest(1,nums)
heapq.nsmallest(1,nums)

[-4]

两个函数都能接受一个关键字参数，用于更复杂的数据结构中：

In [31]:
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price': 91.1},
    {'name': 'AAPL', 'shares': 50, 'price': 543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares': 45, 'price': 16.35},
    {'name': 'ACME', 'shares': 75, 'price': 115.65}
]
cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])

In [32]:
cheap
#上面代码在对每个元素进行对比的时候，会以 price 的值进行比较。

[{'name': 'YHOO', 'shares': 45, 'price': 16.35},
 {'name': 'FB', 'shares': 200, 'price': 21.09},
 {'name': 'HPQ', 'shares': 35, 'price': 31.75}]

如果你想在一个集合中查找最小或最大的 N 个元素，并且 N 小于集合元素数量，那么这些函数提供了很好的性能。 因为在底层实现里面，首先会先将集合数据进行堆排序后放入一个列表中：

In [41]:
nums = [1,2,233,12,-1,-1,-99,32,-90,11.2]
heap = list(nums)
heapq.heapify(heap)
heapq.heappop(heap)

-99

1.5 实现一个优先级队列

问题

怎样实现一个按优先级排序的队列？ 并且在这个队列上面每次 pop 操作总是返回优先级最高的那个元素

解决方案

下面的类利用 heapq 模块实现了一个简单的优先级队列：

In [42]:
import heapq

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0

    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1

    def pop(self):
        return heapq.heappop(self._queue)[-1]

In [43]:
class Item:
    def __init__(self, name):
         self.name = name
    def __repr__(self):
         return 'Item({!r})'.format(self.name)

In [55]:
q = PriorityQueue()
q.push(Item('foo'),1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)
q.pop()

Item('bar')

In [50]:
q.pop()

Item('spam')

In [51]:
q.pop()

Item('foo')

In [52]:
q.pop()

Item('grok')

仔细观察可以发现，第一个 pop() 操作返回优先级最高的元素。 另外注意到如果两个有着相同优先级的元素（ foo 和 grok ），pop 操作按照它们被插入到队列的顺序返回的。

讨论

这一小节我们主要关注 heapq 模块的使用。 函数 heapq.heappush() 和 heapq.heappop() 分别在队列 _queue 上插入和删除第一个元素， 并且队列 _queue 保证第一个元素拥有最高优先级（ 1.4 节已经讨论过这个问题）。 heappop() 函数总是返回”最小的”的元素，这就是保证队列pop操作返回正确元素的关键。 另外，由于 push 和 pop 操作时间复杂度为 O(log N)，其中 N 是堆的大小，因此就算是 N 很大的时候它们运行速度也依旧很快。

在上面代码中，队列包含了一个 (-priority, index, item) 的元组。 优先级为负数的目的是使得元素按照优先级从高到低排序。 这个跟普通的按优先级从低到高排序的堆排序恰巧相反。

index 变量的作用是保证同等优先级元素的正确排序。 通过保存一个不断增加的 index 下标变量，可以确保元素按照它们插入的顺序排序。 而且， index 变量也在相同优先级元素比较的时候起到重要作用。

为了阐明这些，先假定 Item 实例是不支持排序的：

1.6 字典中的键映射多个值

问题

怎样实现一个键对应多个值的字典（也叫 multidict）？

解决方案

一个字典就是一个键对应一个单值的映射。如果你想要一个键映射多个值，那么你就需要将这多个值放到另外的容器中， 比如列表或者集合里面。比如，你可以像下面这样构造这样的字典：



In [57]:
d = {
    'a' : [1, 2, 3],
    'b' : [4, 5]
}
e = {
    'a' : {1, 2, 3},
    'b' : {4, 5}
}

选择使用列表还是集合取决于你的实际需求。如果你想保持元素的插入顺序就应该使用列表， 如果想去掉重复元素就使用集合（并且不关心元素的顺序问题）。

你可以很方便的使用 collections 模块中的 defaultdict 来构造这样的字典。 defaultdict 的一个特征是它会自动初始化每个 key 刚开始对应的值，所以你只需要关注添加元素操作了。比如：

In [58]:
from collections import defaultdict

d = defaultdict(list)
d['a'].append(1)
d['a'].append(2)
d['b'].append(4)

d = defaultdict(set)
d['a'].add(1)
d['a'].add(2)
d['b'].add(4)

需要注意的是， defaultdict 会自动为将要访问的键（就算目前字典中并不存在这样的键）创建映射实体。 如果你并不需要这样的特性，你可以在一个普通的字典上使用 setdefault() 方法来代替。比如：

In [59]:
d = {} # 一个普通的字典
d.setdefault('a', []).append(1)
d.setdefault('a', []).append(2)
d.setdefault('b', []).append(4)

但是很多程序员觉得 setdefault() 用起来有点别扭。因为每次调用都得创建一个新的初始值的实例（例子程序中的空列表 [] ）

一般来讲，创建一个多值映射字典是很简单的。但是，如果你选择自己实现的话，那么对于值的初始化可能会有点麻烦， 你可能会像下面这样来实现：

In [60]:
d = {}
for key, value in pairs:
    if key not in d:
        d[key] = []
    d[key].append(value)

NameError: name 'pairs' is not defined

In [61]:
from collections import OrderedDict

d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4
# Outputs "foo 1", "bar 2", "spam 3", "grok 4"
for key in d:
    print(key, d[key])

foo 1
bar 2
spam 3
grok 4


In [62]:
d

OrderedDict([('foo', 1), ('bar', 2), ('spam', 3), ('grok', 4)])

当你想要构建一个将来需要序列化或编码成其他格式的映射的时候， OrderedDict 是非常有用的。 比如，你想精确控制以 JSON 编码后字段的顺序，你可以先使用 OrderedDict 来构建这样的数据：



In [63]:
import json
json.dumps(d)

'{"foo": 1, "bar": 2, "spam": 3, "grok": 4}'

OrderedDict 内部维护着一个根据键插入顺序排序的双向链表。每次当一个新的元素插入进来的时候， 它会被放到链表的尾部。对于一个已经存在的键的重复赋值不会改变键的顺序。

需要注意的是，一个 OrderedDict 的大小是一个普通字典的两倍，因为它内部维护着另外一个链表。 所以如果你要构建一个需要大量 OrderedDict 实例的数据结构的时候（比如读取 100,000 行 CSV 数据到一个 OrderedDict 列表中去）， 那么你就得仔细权衡一下是否使用 OrderedDict 带来的好处要大过额外内存消耗的影响

In [64]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}


为了对字典值执行计算操作，通常需要使用 zip() 函数先将键和值反转过来。 比如，下面是查找最小和最大股票价格和股票值的代码：

In [66]:
min_price = min(zip(prices.values(),prices.keys()))
min_price

(10.75, 'FB')

In [68]:
price_sorted = sorted(zip(prices.values(),prices.keys()))
price_sorted

[(10.75, 'FB'),
 (37.2, 'HPQ'),
 (45.23, 'ACME'),
 (205.55, 'IBM'),
 (612.78, 'AAPL')]

执行这些计算的时候，需要注意的是 zip() 函数创建的是一个只能访问一次的迭代器。 比如，下面的代码就会产生错误：

In [70]:
prices_and_names = zip(prices.values(), prices.keys())
print(min(prices_and_names))
print(max(prices_and_names))

(10.75, 'FB')


ValueError: max() arg is an empty sequence

In [72]:
#如果你在一个字典上执行普通的数学运算，你会发现它们仅仅作用于键，而不是值。比如
min(prices)

'AAPL'

这个结果并不是你想要的，因为你想要在字典的值集合上执行这些计算。 或许你会尝试着使用字典的 values() 方法来解决这个问题：


In [74]:
prices.values()

dict_values([45.23, 612.78, 205.55, 37.2, 10.75])

In [75]:
#你可以在 min() 和 max() 函数中提供 key 函数参数来获取最小值或最大值对应的键的信息。比如：

min(prices, key=lambda k: prices[k]) # Returns 'FB'
max(prices, key=lambda k: prices[k])



'AAPL'

In [77]:
prices = { 'AAA' : 45.23, 'ZZZ': 45.23 }
min(zip(prices.values(),prices.keys()))

(45.23, 'AAA')

问题

怎样在两个字典中寻寻找相同点（比如相同的键、相同的值等等）？

解决方案

考虑下面两个字典：

In [78]:
a = {
    'x' : 1,
    'y' : 2,
    'z' : 3
}

b = {
    'w' : 10,
    'x' : 11,
    'y' : 2
}

为了寻找两个字典的相同点，可以简单的在两字典的 keys() 或者 items() 方法返回结果上执行集合操作。比如：

In [79]:
a.keys()&b.keys()

{'x', 'y'}

In [81]:
a.items()&b.items()

{('y', 2)}

In [83]:
# 这些操作也可以用于修改或者过滤字典元素。
# 比如，假如你想以现有字典构造一个排除几个指定键的新字典。 
# 下面利用字典推导来实现这样的需求：

# Make a new dictionary with certain keys removed
c = {key:a[key] for key in a.keys() - {'z', 'w'}}
c

{'y': 2, 'x': 1}

删除序列相同元素并保持顺序

如果序列上的值都是 hashable 类型，那么可以很简单的利用集合或者生成器来解决这个问题。比如：



In [84]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [89]:
a = [1,23,13,4,12,45,4,1,6,4,23,45,11]
list(dedupe(a))

[1, 23, 13, 4, 12, 45, 6, 11]

In [90]:
set(a)

{1, 4, 6, 11, 12, 13, 23, 45}

这个方法仅仅在序列中元素为 hashable 的时候才管用。 如果你想消除元素不可哈希（比如 dict 类型）的序列中重复元素的话，你需要将上述代码稍微改变一下，就像这样：



In [91]:
def dedupe(items,key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)

In [94]:
#这里的key参数指定了一个函数，将序列元素转换成 hashable 类型。下面是它的用法示例
a = [ {'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]
list(dedupe(a,key=lambda d:(d['x'],d['y'])))

[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]

In [96]:
record = '....................100 .......513.25 ..........'
cost = int(record[20:23]) * float(record[31:37])
cost

51325.0

In [97]:
SHARES = slice(20, 23)
PRICE = slice(31, 37)
cost = int(record[SHARES]) * float(record[PRICE])

In [98]:
cost

51325.0

In [99]:
'''
collections.Counter 类就是专门为这类问题而设计的， 
它甚至有一个有用的 most_common() 方法直接给了你答案。

为了演示，先假设你有一个单词列表并且想找出哪个单词出现频率最高。你可以这样做：
作为输入， Counter 对象可以接受任意的由可哈希（hashable）元素构成的序列对象。 
在底层实现上，一个 Counter 对象就是一个字典，将元素映射到它出现的次数上。

'''
words = [
    'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
    'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
    'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
    'my', 'eyes', "you're", 'under'
]

from collections import Counter
word_count = Counter(words)
#出现频率最高的三个词
top_three = word_count.most_common(3)
top_three

[('eyes', 8), ('the', 5), ('look', 4)]

In [100]:
morewords = ['吴磊','无','吴磊','没有','有','无','吴磊','吴磊']
for word in morewords:
    word_count[word]+=1

In [101]:
word_count['吴磊']

4

In [102]:
word_count

Counter({'look': 4,
         'into': 3,
         'my': 3,
         'eyes': 8,
         'the': 5,
         'not': 1,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1,
         '吴磊': 4,
         '无': 2,
         '没有': 1,
         '有': 1})

In [103]:
'''
1.13 通过某个关键字排序一个字典列表

问题

你有一个字典列表，你想根据某个或某几个字典字段来排序这个列表。

解决方案

通过使用 operator 模块的 itemgetter 函数，可以非常容易的排序这样的数据结构。 
假设你从数据库中检索出来网站会员信息列表，并且以下列的数据结构返回：

'''
rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
    {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
    {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}
]
#根据任意的字典字段来排序输入结果行是很容易实现的
from operator import itemgetter
rows_by_fname = sorted(rows,key = itemgetter('fname'))
row_by_Uid = sorted(rows,key=itemgetter('uid'))
print(row_by_Uid,rows_by_fname)

[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}] [{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]


In [104]:
rows_by_lfname = sorted(rows, key=itemgetter('lname','fname'))
print(rows_by_lfname)

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]


在上面例子中， rows 被传递给接受一个关键字参数的 sorted() 内置函数。 这个参数是 callable 类型，并且从 rows 中接受一个单一元素，然后返回被用来排序的值。 itemgetter() 函数就是负责创建这个 callable 对象的。

operator.itemgetter() 函数有一个被 rows 中的记录用来查找值的索引参数。可以是一个字典键名称， 一个整形值或者任何能够传入一个对象的 __getitem__() 方法的值。 如果你传入多个索引参数给 itemgetter() ，它生成的 callable 对象会返回一个包含所有元素值的元组， 并且 sorted() 函数会根据这个元组中元素顺序去排序。 但你想要同时在几个字段上面进行排序（比如通过姓和名来排序，也就是例子中的那样）的时候这种方法是很有用的。

itemgetter() 有时候也可以用 lambda 表达式代替

In [106]:
rowsq = sorted(rows,key=lambda r :r['fname'])
rowsq

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
 {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]

这种方案也不错。但是，使用 itemgetter() 方式会运行的稍微快点。因此，如果你对性能要求比较高的话就使用 itemgetter() 方式。

最后，不要忘了这节中展示的技术也同样适用于 min() 和 max() 等函数。比如：

In [107]:
min(rows,key=itemgetter('uid'))

{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}

1.14 排序不支持原生比较的对象

问题

你想排序类型相同的对象，但是他们不支持原生的比较操作。

解决方案

内置的 sorted() 函数有一个关键字参数 key ，可以传入一个 callable 对象给它， 这个 callable 对象对每个传入的对象返回一个值，这个值会被 sorted 用来排序这些对象。 比如，如果你在应用程序里面有一个 User 实例序列，并且你希望通过他们的 user_id 属性进行排序， 你可以提供一个以 User 实例作为输入并输出对应 user_id 值的 callable 对象。比如：

In [110]:
class User:
    def __init__(self,user_id):
        self.user_id = user_id
    def __repr__(self):
        return 'User({})'.format(self.user_id)
def sort_notcompare():
    users = [User(23),User(2),User(99)]
    print(users)
    print(sorted(users, key=lambda u: u.user_id))

In [112]:
#另外一种方式是使用 operator.attrgetter() 来代替 lambda 函数
from operator import attrgetter
users = [User(23),User(2),User(99)]
sorted(users, key=attrgetter('user_id'))

[User(2), User(23), User(99)]

选择使用 lambda 函数或者是 attrgetter() 可能取决于个人喜好。 但是， attrgetter() 函数通常会运行的快点，并且还能同时允许多个字段进行比较。 这个跟 operator.itemgetter() 函数作用于字典类型很类似（参考1.13小节）。 例如，如果 User 实例还有一个 first_name 和 last_name 属性，那么可以向下面这样排序：

In [113]:
by_name = sorted(users, key=attrgetter('last_name', 'first_name'))

AttributeError: 'User' object has no attribute 'last_name'

1.15 通过某个字段将记录分组

问题

你有一个字典或者实例的序列，然后你想根据某个特定的字段比如 date 来分组迭代访问。

解决方案

itertools.groupby() 函数对于这样的数据分组操作非常实用。 为了演示，假设你已经有了下列的字典列表：

In [114]:
rows = [
    {'address': '5412 N CLARK', 'date': '07/01/2012'},
    {'address': '5148 N CLARK', 'date': '07/04/2012'},
    {'address': '5800 E 58TH', 'date': '07/02/2012'},
    {'address': '2122 N CLARK', 'date': '07/03/2012'},
    {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
    {'address': '1060 W ADDISON', 'date': '07/02/2012'},
    {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
    {'address': '1039 W GRANVILLE', 'date': '07/04/2012'},
]

现在假设你想在按 date 分组后的数据块上进行迭代。为了这样做，你首先需要按照指定的字段(这里就是 date )排序， 然后调用 itertools.groupby() 函数：

In [120]:
from operator import itemgetter
from itertools import groupby
# rows.sort(key=itemgetter('date'))
for date,items in groupby(rows,key=itemgetter('date')):
    print(date)
    for i in items:
        print(' ',i)

07/01/2012
  {'address': '5412 N CLARK', 'date': '07/01/2012'}
  {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/02/2012
  {'address': '5800 E 58TH', 'date': '07/02/2012'}
  {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}
  {'address': '1060 W ADDISON', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
  {'address': '5148 N CLARK', 'date': '07/04/2012'}
  {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


In [117]:
rows.sort(key=itemgetter('date'))
rows

[{'address': '5412 N CLARK', 'date': '07/01/2012'},
 {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
 {'address': '5800 E 58TH', 'date': '07/02/2012'},
 {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
 {'address': '1060 W ADDISON', 'date': '07/02/2012'},
 {'address': '2122 N CLARK', 'date': '07/03/2012'},
 {'address': '5148 N CLARK', 'date': '07/04/2012'},
 {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}]

groupby() 函数扫描整个序列并且查找连续相同值（或者根据指定 key 函数返回值相同）的元素序列。 在每次迭代的时候，它会返回一个值和一个迭代器对象， 这个迭代器对象可以生成元素值全部等于上面那个值的组中所有对象。

一个非常重要的准备步骤是要根据指定的字段将数据排序。 因为 groupby() 仅仅检查连续的元素，如果事先并没有排序完成的话，分组函数将得不到想要的结果。

如果你仅仅只是想根据 date 字段将数据分组到一个大的数据结构中去，并且允许随机访问， 那么你最好使用 defaultdict() 来构建一个多值字典，关于多值字典已经在 1.6 小节有过详细的介绍。比如：

In [124]:
from collections import defaultdict
rows_by_date = defaultdict(list)
for row in rows:
    rows_by_date[row['date']].append(row)
rows_by_date

defaultdict(list,
            {'07/01/2012': [{'address': '5412 N CLARK', 'date': '07/01/2012'},
              {'address': '4801 N BROADWAY', 'date': '07/01/2012'}],
             '07/02/2012': [{'address': '5800 E 58TH', 'date': '07/02/2012'},
              {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
              {'address': '1060 W ADDISON', 'date': '07/02/2012'}],
             '07/03/2012': [{'address': '2122 N CLARK', 'date': '07/03/2012'}],
             '07/04/2012': [{'address': '5148 N CLARK', 'date': '07/04/2012'},
              {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}]})

有时候，过滤规则比较复杂，不能简单的在列表推导或者生成器表达式中表达出来。 比如，假设过滤的时候需要处理一些异常或者其他复杂情况。这时候你可以将过滤代码放到一个函数中， 然后使用内建的 filter() 函数。示例如下：

In [125]:
values = ['1', '2', '-3', '-', '4', 'N/A', '5']
def is_int(val):
    try:
        x=int(val)
        return True
    except ValueError:
        return False
ivals = list(filter(is_int, values))
print(ivals)

['1', '2', '-3', '4', '5']


filter() 函数创建了一个迭代器，因此如果你想得到一个列表的话，就得像示例那样使用 list() 去转换。

列表推导和生成器表达式通常情况下是过滤数据最简单的方式。 其实它们还能在过滤的时候转换数据

In [128]:
 mylist = [1, 4, -5, 10, -7, 2, 3, -1]
import math
 [math.sqrt(n) for n in mylist if n > 0]

[1.0, 2.0, 3.1622776601683795, 1.4142135623730951, 1.7320508075688772]

过滤操作的一个变种就是将不符合条件的值用新的值代替，而不是丢弃它们。 比如，在一列数据中你可能不仅想找到正数，而且还想将不是正数的数替换成指定的数。 通过将过滤条件放到条件表达式中去，可以很容易的解决这个问题，就像这样：

In [130]:
clip_neg = [n if n >0 else 0 for n in mylist]
clip_neg

[1, 4, 0, 10, 0, 2, 3, 0]

In [132]:
tet = [n if n <0 else 'm' for n in mylist]
tet

['m', 'm', -5, 'm', -7, 'm', 'm', -1]

另外一个值得关注的过滤工具就是 itertools.compress() ， 它以一个 iterable 对象和一个相对应的 Boolean 选择器序列作为输入参数。 然后输出 iterable 对象中对应选择器为 True 的元素。 当你需要用另外一个相关联的序列来过滤某个序列的时候，这个函数是非常有用的。 比如，假如现在你有下面两列数据：



In [133]:
addresses = [
    '5412 N CLARK',
    '5148 N CLARK',
    '5800 E 58TH',
    '2122 N CLARK',
    '5645 N RAVENSWOOD',
    '1060 W ADDISON',
    '4801 N BROADWAY',
    '1039 W GRANVILLE',
]
counts = [ 0, 3, 10, 4, 1, 7, 6, 1]

In [136]:
#现在你想将那些对应 count 值大于5的地址全部输出，那么你可以这样做
from itertools import compress
more4= [n>5 for n in counts]
more4

[False, False, True, False, False, True, True, False]

In [137]:
list(compress(addresses,more4))

['5800 E 58TH', '1060 W ADDISON', '4801 N BROADWAY']

这里的关键点在于先创建一个 Boolean 序列，指示哪些元素符合条件。 然后 compress() 函数根据这个序列去选择输出对应位置为 True 的元素。

和 filter() 函数类似， compress() 也是返回的一个迭代器。因此，如果你需要得到一个列表， 那么你需要使用 list() 来将结果转换为列表类型。

1.17 从字典中提取子集

问题

你想构造一个字典，它是另外一个字典的子集。

解决方案

最简单的方式是使用字典推导。比如：



In [138]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}
# Make a dictionary of all prices over 200
p1 = {key: value for key, value in prices.items() if value > 200}
# Make a dictionary of tech stocks
tech_names = {'AAPL', 'IBM', 'HPQ', 'MSFT'}
p2 = {key: value for key, value in prices.items() if key in tech_names}

In [140]:
print(p1,p2)

{'AAPL': 612.78, 'IBM': 205.55} {'AAPL': 612.78, 'IBM': 205.55, 'HPQ': 37.2}


In [142]:
p1 = dict((key, value) for key, value in prices.items() if value > 200)
p1

{'AAPL': 612.78, 'IBM': 205.55}

但是，字典推导方式表意更清晰，并且实际上也会运行的更快些 （在这个例子中，实际测试几乎比 dict() 函数方式快整整一倍）。

In [143]:
tech_names = { 'AAPL', 'IBM', 'HPQ', 'MSFT' }
p2 = { key:prices[key] for key in prices.keys() & tech_names }

1.18 映射名称到序列元素

问题

你有一段通过下标访问列表或者元组中元素的代码，但是这样有时候会使得你的代码难以阅读， 于是你想通过名称来访问元素。

解决方案

collections.namedtuple() 函数通过使用一个普通的元组对象来帮你解决这个问题。 这个函数实际上是一个返回 Python 中标准元组类型子类的一个工厂方法。 你需要传递一个类型名和你需要的字段给它，然后它就会返回一个类，你可以初始化这个类，为你定义的字段传递值等。 代码示例：

In [145]:
from collections import namedtuple
Subscriber = namedtuple('Subscriber', ['addr', 'joined'])
sub = Subscriber('jonesy@example.com', '2012-10-19')
sub

Subscriber(addr='jonesy@example.com', joined='2012-10-19')

尽管 namedtuple 的实例看起来像一个普通的类实例，但是它跟元组类型是可交换的，支持所有的普通元组操作，比如索引和解压。 比如：

In [146]:
len(sub)

2

命名元组的一个主要用途是将你的代码从下标操作中解脱出来。 因此，如果你从数据库调用中返回了一个很大的元组列表，通过下标去操作其中的元素， 当你在表中添加了新的列的时候你的代码可能就会出错了。但是如果你使用了命名元组，那么就不会有这样的顾虑。

为了说明清楚，下面是使用普通元组的代码：

In [147]:
def computer_cost(records):
    total = 0.0
    for rec in records:
        total += rec[1]*rec[2]
    return total

下标操作通常会让代码表意不清晰，并且非常依赖记录的结构。 下面是使用命名元组的版本：

In [148]:
from collections import namedtuple
Stock = namedtuple('Stock', ['name', 'shares', 'price'])
def compute_cost(records):
    total = 0.0
    for rec in records:
        s = Stock(*rec)
        total += s.shares * s.price
    return total

命名元组另一个用途就是作为字典的替代，因为字典存储需要更多的内存空间。 如果你需要构建一个非常大的包含字典的数据结构，那么使用命名元组会更加高效。 但是需要注意的是，不像字典那样，一个命名元组是不可更改的。比如：

In [150]:
s = Stock('ACME', 100, 123.45)
s

Stock(name='ACME', shares=100, price=123.45)

如果你真的需要改变属性的值，那么可以使用命名元组实例的 _replace() 方法， 它会创建一个全新的命名元组并将对应的字段用新的值取代。比如

In [151]:
s = s._replace(shares=77)
s

Stock(name='ACME', shares=77, price=123.45)

_replace() 方法还有一个很有用的特性就是当你的命名元组拥有可选或者缺失字段时候， 它是一个非常方便的填充数据的方法。 你可以先创建一个包含缺省值的原型元组，然后使用 _replace() 方法创建新的值被更新过的实例。比如：

In [152]:
from collections import namedtuple

Stock = namedtuple('Stock', ['name', 'shares', 'price', 'date', 'time'])

# Create a prototype instance
stock_prototype = Stock('', 0, 0.0, None, None)

# Function to convert a dictionary to a Stock
def dict_to_stock(s):
    return stock_prototype._replace(**s)

In [153]:
#下面是调用
a = {'name': 'ACME', 'shares': 100, 'price': 123.45}
dict_to_stock(a)

Stock(name='ACME', shares=100, price=123.45, date=None, time=None)

In [154]:
 b = {'name': 'ACME', 'shares': 100, 'price': 123.45, 'date': '12/17/2012'}
b

{'name': 'ACME', 'shares': 100, 'price': 123.45, 'date': '12/17/2012'}

In [157]:
nums = [1,2,3,4,44]
aa = [x**2 for x in nums]
aa

[1, 4, 9, 16, 1936]

In [159]:
# Determine if any .py files exist in a directory
import os
# files = os.listdir('dirname')
# if any(name.endswith('.py') for name in files):
#     print('There be python!')
# else:
#     print('Sorry, no python.')
# Output a tuple as CSV
s = ('ACME', 50, 123.45)
print(','.join(str(x) for x in s))
# Data reduction across fields of a data structure
portfolio = [
    {'name':'GOOG', 'shares': 50},
    {'name':'YHOO', 'shares': 75},
    {'name':'AOL', 'shares': 20},
    {'name':'SCOX', 'shares': 65}
]
min_shares = min(s['shares'] for s in portfolio)

ACME,50,123.45


In [162]:
s = sum((x * x for x in nums))
s

1966

In [164]:
nums = [1, 2, 3, 4, 5]
s = sum([x * x for x in nums])
s

55

这种方式同样可以达到想要的效果，但是它会多一个步骤，先创建一个额外的列表。 对于小型列表可能没什么关系，但是如果元素数量非常大的时候， 它会创建一个巨大的仅仅被使用一次就被丢弃的临时数据结构。而生成器方案会以迭代的方式转换数据，因此更省内存。

在使用一些聚集函数比如 min() 和 max() 的时候你可能更加倾向于使用生成器版本， 它们接受的一个 key 关键字参数或许对你很有帮助。 比如，在上面的证券例子中，你可能会考虑下面的实现版本：



In [169]:
min_shares = min(s['shares'] for s in portfolio)
min_shares
min_shares1 = min(portfolio,key=lambda s :s['shares'])
min_shares1

{'name': 'AOL', 'shares': 20}

1.20 合并多个字典或映射

问题

现在有多个字典或者映射，你想将它们从逻辑上合并为一个单一的映射后执行某些操作， 比如查找值或者检查某些键是否存在。

解决方案

假如你有如下两个字典:

In [170]:
a = {'x': 1, 'z': 3 }
b = {'y': 2, 'z': 4 }

现在假设你必须在两个字典中执行查找操作（比如先从 a 中找，如果找不到再在 b 中找）。 一个非常简单的解决方案就是使用 collections 模块中的 ChainMap 类。比如：

In [173]:
from collections import ChainMap
c = ChainMap(a,b)
print(c['x'])
print(c)

1
ChainMap({'x': 1, 'z': 3}, {'y': 2, 'z': 4})


一个 ChainMap 接受多个字典并将它们在逻辑上变为一个字典。 然后，这些字典并不是真的合并在一起了， ChainMap 类只是在内部创建了一个容纳这些字典的列表 并重新定义了一些常见的字典操作来遍历这个列表。大部分字典操作都是可以正常使用的，比如：



In [175]:
len(c)

3

In [176]:
list(c.keys())

['z', 'y', 'x']

In [177]:
list(c.values())

[3, 2, 1]

如果出现重复键，那么第一次出现的映射值会被返回。 因此，例子程序中的 c['z'] 总是会返回字典 a 中对应的值，而不是 b 中对应的值。

对于字典的更新或删除操作总是影响的是列表中第一个字典。比如：



In [178]:
c['z']=10

In [179]:
c

ChainMap({'x': 1, 'z': 10}, {'y': 2, 'z': 4})

In [181]:
c['w']=101
c

ChainMap({'x': 1, 'z': 10, 'w': 101}, {'y': 2, 'z': 4})

In [185]:
del c['z']


In [186]:
c

ChainMap({'w': 101}, {'y': 2, 'z': 4})

ChainMap 对于编程语言中的作用范围变量（比如 globals , locals 等）是非常有用的。 事实上，有一些方法可以使它变得简单：

In [189]:
values = ChainMap()
values['x']=1
values

ChainMap({'x': 1})

In [190]:
values = values.new_child()

In [191]:
values['x']=2
values

ChainMap({'x': 2}, {'x': 1})

In [193]:
values = values.new_child()
values['x'] = 3
values

ChainMap({'x': 3}, {'x': 3}, {'x': 2}, {'x': 1})

In [194]:
values['x']

3

In [196]:
values = values.parents
values['x']

2

In [197]:
values

ChainMap({'x': 2}, {'x': 1})

ChainMap 使用原来的字典，它自己不创建新的字典。所以它并不会产生上面所说的结果，比如：

In [199]:
a = {'x': 1, 'z': 3 }
b = {'y': 2, 'z': 4 }
merged = ChainMap(a, b)

In [200]:
merged['x']

1

In [201]:
a['x']=32
merged['x']

32

In [202]:
merged

ChainMap({'x': 32, 'z': 3}, {'y': 2, 'z': 4})